# Import Libraries

In [242]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

# Data Loading

In [144]:
content_based_dataframe = pd.read_csv("./preprocessed_dataset/content_based_dataframe.csv")
# load all dataset
anime_df_merged = pd.read_csv('./preprocessed_dataset/anime_df_merged.csv')
anime_with_synopsis_df = pd.read_csv('./preprocessed_dataset/anime_with_synopsis.csv')
rating_df_lite_version = pd.read_csv('./preprocessed_dataset/rating_df_lite_version.csv')

# Modelling

## Content-based filtering

In [3]:
content_based_dataframe.head()

,MAL_ID,Genre_Action,Genre_Adventure,Genre_Cars,Genre_Comedy,Genre_Dementia,Genre_Demons,Genre_Drama,Genre_Ecchi,Genre_Fantasy,...,Type_OVA,Type_Special,Type_TV,Episodes,year_premiered,start_date_aired,start_year_aired,duration_hours,duration_minutes,duration_seconds
0,0,1,1,0,1,0,0,1,0,0,...,0,0,1,0.008181,0.616667,0.066667,0.778846,0.0,0.406780,0.0
1,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0.000000,0.916667,0.000000,0.807692,0.5,0.932203,0.0
2,2,1,1,0,1,0,0,1,0,0,...,0,0,1,0.008181,0.616667,0.000000,0.778846,0.0,0.406780,0.0
3,3,1,0,0,0,0,0,1,0,0,...,0,0,1,0.008181,0.683333,0.033333,0.817308,0.0,0.423729,0.0
4,4,0,1,0,0,0,0,0,0,1,...,0,0,1,0.016688,0.716667,0.966667,0.836538,0.0,0.389831,0.0


In [4]:
# drop MAL_ID information
X = content_based_dataframe.drop('MAL_ID', axis=1)

In [9]:
X

,Genre_Action,Genre_Adventure,Genre_Cars,Genre_Comedy,Genre_Dementia,Genre_Demons,Genre_Drama,Genre_Ecchi,Genre_Fantasy,Genre_Game,...,Type_OVA,Type_Special,Type_TV,Episodes,year_premiered,start_date_aired,start_year_aired,duration_hours,duration_minutes,duration_seconds
0,1,1,0,1,0,0,1,0,0,0,...,0,0,1,0.008181,0.616667,0.066667,0.778846,0.0,0.406780,0.0
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0.000000,0.916667,0.000000,0.807692,0.5,0.932203,0.0
2,1,1,0,1,0,0,1,0,0,0,...,0,0,1,0.008181,0.616667,0.000000,0.778846,0.0,0.406780,0.0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0.008181,0.683333,0.033333,0.817308,0.0,0.423729,0.0
4,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0.016688,0.716667,0.966667,0.836538,0.0,0.389831,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,1,1,0,0,0,0,0,0,1,1,...,0,0,0,0.005563,0.916667,0.266667,0.951923,0.0,0.067797,0.0
12075,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0.002618,0.916667,0.800000,0.932692,0.0,0.067797,0.0
12076,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0.003599,0.933333,0.300000,0.961538,0.0,0.389831,0.0
12077,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0.003599,0.933333,0.266667,0.961538,0.0,0.389831,0.0


### Similarity and Distance Based

In [47]:
def anime_recommendation_similarity(anime_title, similarity_matrix_df, items_df, k=5):
    """
    Deskripsi:
    Fungsi untuk mencari top-k rekomendasi dari judul anime yang diberikan berdasarkan matriks similarity yang dimasukkan
    
    Input:
    - anime_title: 
        - <type>: string
        - judul anime yang akan dicari rekomendasinya berdasarkan kemiripan pada similarity_matrix_df
    - similarity_matrix_df
        - <type>: dataframe
        - matrix similarity atau distance matrix
    - items_df
        - <type>: dataframe
        - dataframe data anime_df_merged
    - k
        - <type>: integer
        - Jumlah rekomendasi yang ingin ditampilkan
    """
    k=5
    name='Cowboy Bebop'
    
    # cari k- nama anime (index yang memiliki scor similarity terbesar dengan nama anime yang dicari)
    index=cosine_sim_df.loc[:,name].sort_values(ascending=False).drop(name).head(k).index
    
    # cari data dari items_df dengan index
    df_recommendation = items_df[
        items_df['Name'].isin(index)
    ].copy()
    
    return df_recommendation

#### Cosine Similarity

In [27]:
# hitung nilai cosine similarity dari data content-based filtering
cosine_sim = cosine_similarity(X) 
cosine_sim

array([[1.        , 0.59611146, 0.57595368, ..., 0.31155693, 0.26091885,
        0.37255639],
       [0.59611146, 1.        , 0.31804479, ..., 0.222663  , 0.11387199,
        0.34447892],
       [0.57595368, 0.31804479, 1.        , ..., 0.31047336, 0.31790945,
        0.28914456],
       ...,
       [0.31155693, 0.222663  , 0.31047336, ..., 1.        , 0.18713945,
        0.27772653],
       [0.26091885, 0.11387199, 0.31790945, ..., 0.18713945, 1.        ,
        0.21789173],
       [0.37255639, 0.34447892, 0.28914456, ..., 0.27772653, 0.21789173,
        1.        ]])

In [6]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa Nama anime
cosine_sim_df = pd.DataFrame(cosine_sim, index=anime_df_merged['Name'], columns=anime_df_merged['Name'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap resto
cosine_sim_df.sample(15, axis=1).sample(10, axis=0)

Shape: (12079, 12079)


Name,Hadakya no Otonosama,Cream Lemon,Timing,Diamond no Ace,Noragami OVA,Tenchi Muyou! GXP,Nazo no Kanojo X: Nazo no Natsu Matsuri,Robotan (1986),Shooting Hero,Digimon Xros Wars,Black Lagoon Omake,Dambo,Kirameki no Kyoudai: Sex Crime,Pokemon Housoukyoku,Taiko no Tatsujin
Name,,,,,,,,,,,,,,,
Romantica Clock,0.502442,0.200415,0.504136,0.375583,0.339302,0.207672,0.458880,0.572885,0.471805,0.247932,0.530204,0.398370,0.517264,0.142825,0.644482
Ippou Nippon Mukashibanashi,0.475662,0.418766,0.470969,0.280435,0.319065,0.362306,0.339360,0.435119,0.546447,0.595681,0.389878,0.686393,0.484193,0.518525,0.387734
Chibikko Remi to Meiken Kapi,0.611558,0.253699,0.452551,0.252804,0.384378,0.171871,0.420070,0.509059,0.498281,0.294905,0.372636,0.434948,0.582792,0.291787,0.549244
Ashita no Eleven-tachi,0.583781,0.193039,0.486950,0.361259,0.325305,0.191800,0.448165,0.563393,0.668903,0.229770,0.512058,0.486634,0.622785,0.224482,0.620719
Raja Maharaja,0.632992,0.245572,0.339919,0.179186,0.207232,0.253375,0.310137,0.508550,0.902689,0.294352,0.254503,0.745648,0.452004,0.296552,0.459439
Dororo Pilot,0.493332,0.216876,0.443104,0.377571,0.540103,0.376075,0.498336,0.355096,0.360513,0.257955,0.674715,0.507943,0.569273,0.246393,0.403386
Malice@Doll,0.250918,0.411490,0.672253,0.202082,0.334007,0.340874,0.455193,0.325398,0.411765,0.317276,0.313275,0.453059,0.523177,0.134508,0.356514
Zero Tester: Chikyuu wo Mamore!,0.359394,0.212209,0.313729,0.354817,0.501233,0.520765,0.378402,0.379378,0.248183,0.334473,0.423020,0.374899,0.436523,0.338616,0.389018
Hokuto no Ken,0.172895,0.199492,0.354545,0.273264,0.310906,0.355387,0.254396,0.356741,0.157396,0.400029,0.186516,0.176181,0.288684,0.162215,0.376412


##### Testing mendapatkan rekomendasi

In [28]:
# menampilkan 5 rekomendasi dari anime berjudul 'Cowboy Bebop'
k=5
name='Cowboy Bebop'

In [29]:
anime_df_merged[
    anime_df_merged['Name'] == name
]

,MAL_ID,Name,Genres,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity)


Terlihat bahwa **Cowboy Bebop** memiliki karakteristik seperti di atas, seperti:
- Genre: Action, Adventure, Comedy, Drama, Sci-Fi, Space
- Type: TV
- Producers: Bandai Visual
- dsb

Seharusnya rekomendasi yang diberikan mendekati karakteristik di atas

In [32]:
anime_recommendation_similarity(name, cosine_sim_df, anime_df_merged)

,MAL_ID,Name,Genres,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating
1114,1226,Seihou Tenshi Angel Links,"Action, Adventure, Space, Comedy, Romance, Dra...",TV,13,"Apr 7, 1999 to Jun 30, 1999",Spring 1999,NHK,"Funimation, Bandai Entertainment",Sunrise,Novel,24 min. per ep.,PG-13 - Teens 13 or older
1784,1962,Chikyuu Bouei Kazoku,"Action, Sci-Fi, Comedy, Drama",TV,13,"Jan 9, 2001 to Mar 29, 2001",Winter 2001,Bandai Visual,Funimation,Group TAC,Original,23 min. per ep.,PG-13 - Teens 13 or older
3377,3927,Mobile Suit Gundam 00 Second Season,"Action, Drama, Mecha, Military, Sci-Fi, Space",TV,25,"Oct 5, 2008 to Mar 29, 2009",Fall 2008,Mainichi Broadcasting System,Bandai Entertainment,Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity)
3447,4037,Cowboy Bebop: Yose Atsume Blues,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Special,1,"Jun 26, 1998",Fall 2016,NHK,Funimation,Sunrise,Original,27 min.,R - 17+ (violence & profanity)
10766,31251,Mobile Suit Gundam: Iron-Blooded Orphans,"Action, Drama, Mecha, Sci-Fi, Space",TV,25,"Oct 4, 2015 to Mar 27, 2016",Fall 2015,"Sotsu, Asatsu DK, Mainichi Broadcasting System",Funimation,Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity)


Hasil rekomendasi di atas menunjukkan daftar anime yang mirip dengan **Cowboy Bebop**. Genres dari anime-anime tersebut juga serupa dengan Genres yang dimiliki oleh **Cowboy Bebop**

In [43]:
# menampilkan contoh rekomendasi dari anime lainnya
k=5
name='Seihou Tenshi Angel Links'

In [44]:
anime_df_merged[
    anime_df_merged['Name'] == name
]

,MAL_ID,Name,Genres,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating
1114,1226,Seihou Tenshi Angel Links,"Action, Adventure, Space, Comedy, Romance, Dra...",TV,13,"Apr 7, 1999 to Jun 30, 1999",Spring 1999,NHK,"Funimation, Bandai Entertainment",Sunrise,Novel,24 min. per ep.,PG-13 - Teens 13 or older


In [46]:
# mendapatkan rekomendasi yang mirip dengan 'Seihou Tenshi Angel Links'
anime_recommendation_similarity(name, cosine_sim_df, anime_df_merged)

,MAL_ID,Name,Genres,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating
1114,1226,Seihou Tenshi Angel Links,"Action, Adventure, Space, Comedy, Romance, Dra...",TV,13,"Apr 7, 1999 to Jun 30, 1999",Spring 1999,NHK,"Funimation, Bandai Entertainment",Sunrise,Novel,24 min. per ep.,PG-13 - Teens 13 or older
1784,1962,Chikyuu Bouei Kazoku,"Action, Sci-Fi, Comedy, Drama",TV,13,"Jan 9, 2001 to Mar 29, 2001",Winter 2001,Bandai Visual,Funimation,Group TAC,Original,23 min. per ep.,PG-13 - Teens 13 or older
3377,3927,Mobile Suit Gundam 00 Second Season,"Action, Drama, Mecha, Military, Sci-Fi, Space",TV,25,"Oct 5, 2008 to Mar 29, 2009",Fall 2008,Mainichi Broadcasting System,Bandai Entertainment,Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity)
3447,4037,Cowboy Bebop: Yose Atsume Blues,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Special,1,"Jun 26, 1998",Fall 2016,NHK,Funimation,Sunrise,Original,27 min.,R - 17+ (violence & profanity)
10766,31251,Mobile Suit Gundam: Iron-Blooded Orphans,"Action, Drama, Mecha, Sci-Fi, Space",TV,25,"Oct 4, 2015 to Mar 27, 2016",Fall 2015,"Sotsu, Asatsu DK, Mainichi Broadcasting System",Funimation,Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity)


## Collaborative-based filtering
***
- Teknik ini merekomendasikan item yang mirip dengan preferensi pengguna di masa lalu.
- Teknik ini membutuhkan data rating dari user.
- Goal proyek kita kali ini adalah menghasilkan rekomendasi sejumlah restoran yang sesuai dengan preferensi pengguna berdasarkan rating yang telah diberikan sebelumnya. 
- Dari data rating pengguna, kita akan mengidentifikasi restoran-restoran yang mirip dan belum pernah dikunjungi oleh pengguna untuk direkomendasikan.

In [230]:
df = pd.read_csv("./preprocessed_dataset/collaborative_based_dataframe.csv")
# check data collaborative yang kita punya
df = collaborative_based_dataframe.sample(frac=1, random_state=42)
df = df.sample(8000, random_state=45)
df

,user_id,rating,MAL_ID,Episodes,year_premiered,start_date_aired,start_year_aired,duration_hours,duration_minutes,duration_seconds
5049394,55128,0.888889,1411.0,0.016034,0.750000,0.200000,0.855769,0.0,0.423729,0.0
5418847,58347,0.888889,13.0,0.008181,0.716667,0.133333,0.836538,0.0,0.389831,0.0
4540738,49905,0.777778,2074.0,0.000000,0.916667,0.000000,0.778846,0.5,0.576271,0.0
1553849,17607,0.888889,165.0,0.007526,0.650000,0.600000,0.798077,0.0,0.389831,0.0
1045109,11499,0.888889,4627.0,0.003927,0.816667,0.066667,0.894231,0.0,0.406780,0.0
...,...,...,...,...,...,...,...,...,...,...
378480,4541,0.777778,4418.0,0.031414,0.800000,0.133333,0.884615,0.0,0.389831,0.0
3937571,43897,0.333333,9258.0,0.003599,0.900000,0.266667,0.942308,0.0,0.389831,0.0
4903306,53821,1.000000,3530.0,0.007526,0.783333,0.066667,0.875000,0.0,0.406780,0.0
529924,5905,0.777778,8407.0,0.000000,0.916667,0.266667,0.942308,0.5,0.898305,0.0


In [231]:
# user
user_ids = df["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = df["MAL_ID"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

df["user"] = df["user_id"].map(user2user_encoded)
df["movie"] = df["MAL_ID"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)

In [232]:
print(
    "Number of users: {}, Number of Movies: {}".format(
        num_users, num_movies
    )
)

Number of users: 6801, Number of Movies: 2428


In [233]:
print("Max User ID:", df['user'].max())
print("Max Anime ID:", df['movie'].max())
print("Min User ID:", df['user'].min())
print("Min Anime ID:", df['movie'].min())

Max User ID: 6800
Max Anime ID: 2427
Min User ID: 0
Min Anime ID: 0


In [235]:
# ambil variabel X dan y
X = df[['user','movie']].values
y = df['rating'].values

# Membagi menjadi 80% data train dan 20% data validasi
train_indices = int(0.6 * df.shape[0])
x_train, x_val, y_train, y_val = (
    X[:train_indices],
    X[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

In [238]:
class RecommenderNet(tf.keras.Model):

    # Insialisasi fungsi
    def __init__(self, num_users, num_anime, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_anime = num_anime
        self.embedding_size = embedding_size
        
        self.user_embedding = layers.Embedding(  # layer embedding user
            num_users,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=keras.regularizers.l2(1e-6)
        )
        self.user_bias = layers.Embedding(
            num_users, 1)  # layer embedding user bias
        
        self.anime_embedding = layers.Embedding(  # layer embeddings resto
            num_anime,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=keras.regularizers.l2(1e-6)
        )
        self.anime_bias = layers.Embedding(
            num_anime, 1)  # layer embedding resto bias

    def call(self, inputs):
        user_vector = self.user_embedding(
            inputs[:, 0])  # memanggil layer embedding 1
        user_bias = self.user_bias(inputs[:, 0])  # memanggil layer embedding 2
        anime_vector = self.anime_embedding(
            inputs[:, 1])  # memanggil layer embedding 3
        # memanggil layer embedding 4
        anime_bias = self.anime_bias(inputs[:, 1])

        dot_user_anime = tf.tensordot(user_vector, anime_vector, 2)

        x = dot_user_anime + user_bias + anime_bias

        return tf.nn.sigmoid(x)  # activation sigmoid

In [239]:
model = RecommenderNet(num_users, num_anime, 50) # inisialisasi model
 
# model compile
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [240]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(x_val, y_val),
)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'recommender_net_9/embedding_38/embedding_lookup' defined at (most recent call last):
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado

### Train-validation split

In [204]:
# acak dataframe dulu supaya distribusinya menjadi random
df_sampled = collaborative_based_dataframe.sample(frac=1, random_state=42)

# ambil hanya beberapa sampel dulu untuk percobaan
df_sampled = collaborative_based_dataframe.head(2000)

In [205]:
# user_ids = df_sampled['user_id'].unique()
# anime_ids = df_sampled['MAL_ID'].unique()

# # Create a mapping for user_id
# user_encoded_to_user = {user_id: idx for idx, user_id in enumerate(user_ids)}
# anime_encoded_to_anime = {anime_id: idx for idx, anime_id in enumerate(anime_ids)}

In [213]:
# Encode your training data
x_train[:, 0] = [user_encoded_to_user[user_id] for user_id in x_train[:, 0]]
x_train[:, 1] = [anime_encoded_to_anime[anime_id] for anime_id in x_train[:, 1]]

In [215]:
len(user_encoded_to_user)

24

In [216]:
len(anime_encoded_to_anime)

1130

### Proses Training
***
- Model menghitung skor kecocokan antara pengguna dan anime dengan teknik embedding
    - Pertama, dilakukan proses embedding terhadap user dan anime
    - Kedua, lakukan operasi dot product antara embedding user dan anime
    - Ketiga, tambahkan bias untuk setiap user dan anime
    - Keempat, skor kecocokan ditetapkan dalam skala [0,1] dengan fungsi aktivasi sigmoid
- Model Machine Learning yang digunakan adalah **RecommenderNet** dari keras Model

In [217]:
num_users = len(user_encoded_to_user)
num_anime = len(anime_encoded_to_anime)

In [218]:
class RecommenderNet(tf.keras.Model):

    # Insialisasi fungsi
    def __init__(self, num_users, num_anime, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_anime = num_anime
        self.embedding_size = embedding_size
        
        self.user_embedding = layers.Embedding(  # layer embedding user
            num_users,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=keras.regularizers.l2(1e-6)
        )
        self.user_bias = layers.Embedding(
            num_users, 1)  # layer embedding user bias
        
        self.anime_embedding = layers.Embedding(  # layer embeddings resto
            num_anime,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=keras.regularizers.l2(1e-6)
        )
        self.anime_bias = layers.Embedding(
            num_anime, 1)  # layer embedding resto bias

    def call(self, inputs):
        user_vector = self.user_embedding(
            inputs[:, 0])  # memanggil layer embedding 1
        user_bias = self.user_bias(inputs[:, 0])  # memanggil layer embedding 2
        anime_vector = self.anime_embedding(
            inputs[:, 1])  # memanggil layer embedding 3
        # memanggil layer embedding 4
        anime_bias = self.anime_bias(inputs[:, 1])

        dot_user_anime = tf.tensordot(user_vector, anime_vector, 2)

        x = dot_user_anime + user_bias + anime_bias

        return tf.nn.sigmoid(x)  # activation sigmoid

In [219]:
model = RecommenderNet(num_users, num_anime, 50) # inisialisasi model
 
# model compile
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [220]:
# Memulai training
 
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 8,
    epochs = 10,
    validation_data = (x_val, y_val)
)

Epoch 1/10
180/200 [==========================>...] - ETA: 0s - loss: 0.6902 - root_mean_squared_error: 0.2639

InvalidArgumentError: Graph execution error:

Detected at node 'recommender_net_7/embedding_31/embedding_lookup' defined at (most recent call last):
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\tornado

In [221]:
anime_encoded_to_anime[3554]

224

# Junk Codes

In [8]:


# # Initialize the TfidfVectorizer with comma as separator
# vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(', '))

# # Fit and transform the 'Genres' column
# tfidf_matrix = vectorizer.fit_transform(anime_df_merged['Genres'])

# # Convert the TF-IDF matrix to a DataFrame
# tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# # Combine the MAL_ID with the TF-IDF genre features
# df_with_tfidf_genres = pd.concat([anime_df_merged[['MAL_ID']], tfidf_df], axis=1)

# # Display the DataFrame with TF-IDF scores for each genre
# df_with_tfidf_genres.head()


# # menampilkan nilai TF-IDF dari genres pada setiap judul anime
# pd.DataFrame(
#     tfidf_matrix.todense(), 
#     columns=vectorizer.get_feature_names_out(),
#     index=anime_df_merged.Name
# ).sample(22, axis=1).sample(10, axis=0)